In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-tpu.json'

In [2]:
from glob import glob
import tensorflow as tf
from tqdm import tqdm
import malaya_speech
from malaya_speech.utils import subword
import numpy as np
import mp
from google.cloud import storage

/home/ubuntu/.local/lib/python3.7/site-packages/malaya_boilerplate/frozen_graph.py:25: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [3]:
singlish = glob('part4-same-room/wav/*.wav')
len(singlish)

1148098

In [4]:
subwords = subword.load('transducer-singlish.subword')

In [5]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    string = unicodedata.normalize('NFC', string.lower())
    string = string.replace('\'', '')
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [6]:
def get_after_mandarin(word):
    if '<mandarin>' in word:
        w = word.split('>')[1].split(':')[1]
        return w.split('</')[0]
    else:
        return word
    
def get_before_mandarin(word):
    if '</mandarin>' in word:
        return word.split('</')[0]
    else:
        return word

def replace_paralinguistic(string, replaces = ['(ppb)', '(ppc)', '(ppl)', '(ppo)', '<UNK>', '<MANDARIN>']):
    for r in replaces:
        string = string.replace(r, ' ')
    string = string.split()
    string = [get_after_mandarin(w) for w in string]
    string = [get_before_mandarin(w) for w in string]
    string = [w for w in string if w[0] not in '<[(' and w[-1] not in '>])']
    return ' '.join(string)

In [7]:
def loop(files):
    files, index = files
    results = []
    for i in tqdm(files):
        try:
            p = i.replace('/wav','/text')
            with open(f'{p}.txt') as fopen:
                text = fopen.read()
            if len(text) < 2:
                continue
            if text[0] == '<' and text[-1] == '>':
                continue
            text = replace_paralinguistic(text)
            text = preprocessing_text(text)
            if len(text):
                results.append((i, text))
        except Exception as e:
            pass
    return results

In [8]:
loop((singlish[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 1033.64it/s]


[('part4-same-room/wav/sur_0096_1192_phns_cs-chn-0-223.wav',
  'i can tell you you dont say first you let me guess so when youre upset right i do one of three things first'),
 ('part4-same-room/wav/sur_0525_2050_phns_cs-chn-0-1037.wav',
  'mei you ta shi zhe yang ben lai shi yi zheng pan de ta hui sa de')]

In [9]:
singlishs = mp.multiprocessing(singlish, loop, cores = 12)

100%|██████████| 95674/95674 [05:13<00:00, 305.44it/s] 


In [12]:
len(singlishs)

377265

In [13]:
import six

def to_example(dictionary):
    """Helper: build tf.Example from (string -> int/float/str list) dictionary."""
    features = {}
    for (k, v) in six.iteritems(dictionary):
        if not v:
            raise ValueError('Empty generated field: %s' % str((k, v)))
        # Subtly in PY2 vs PY3, map is not scriptable in py3. As a result,
        # map objects will fail with TypeError, unless converted to a list.
        if six.PY3 and isinstance(v, map):
            v = list(v)
        if isinstance(v[0], six.integer_types) or np.issubdtype(
            type(v[0]), np.integer
        ):
            features[k] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=v)
            )
        elif isinstance(v[0], float):
            features[k] = tf.train.Feature(
                float_list=tf.train.FloatList(value=v)
            )
        elif isinstance(v[0], six.string_types):
            if not six.PY2:  # Convert in python 3.
                v = [bytes(x, 'utf-8') for x in v]
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        elif isinstance(v[0], bytes):
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        else:
            raise ValueError(
                'Value for %s is not a recognized type; v: %s type: %s'
                % (k, str(v[0]), str(type(v[0])))
            )
    return tf.train.Example(features=tf.train.Features(feature=features))

In [14]:
sr = 16000
maxlen = 18
minlen_text = 1
global_count = 0

In [15]:
def loop(files):
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    files, index = files
    output_file = f'{index}-{global_count}.tfrecord'
    writer = tf.io.TFRecordWriter(output_file)
    for s in tqdm(files):
        try:
            if len(s[1]) < minlen_text:
                continue
            y, _ = malaya_speech.load(s[0])
            if (len(y) / sr) > maxlen:
                continue
            t = subword.encode(subwords, s[1], add_blank=False)
            example = to_example({'waveforms': y.tolist(), 
                                  'targets': t, 
                                  'targets_length': [len(t)]})
            writer.write(example.SerializeToString())
        except Exception as e:
            print(e)
            pass
    writer.close()
    blob = bucket.blob(f'imda/part4-same-room/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [17]:
loop((singlishs[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 32.13it/s]


In [18]:
batch_size = 25000
for i in range(0, len(singlishs), batch_size):
    batch = singlishs[i: i + batch_size]
    mp.multiprocessing(batch, loop, cores = 6, returned = False)
    global_count += 1

  3%|▎         | 115/4166 [00:09<06:44, 10.00it/s]

zero-size array to reduction operation maximum which has no identity


 74%|███████▍  | 3073/4166 [02:15<00:39, 27.51it/s]

zero-size array to reduction operation maximum which has no identity


 75%|███████▍  | 3115/4166 [02:15<00:51, 20.25it/s]

...

 79%|███████▊  | 3279/4166 [02:09<00:46, 19.00it/s]

zero-size array to reduction operation maximum which has no identity


  2%|▏         | 6/377 [00:00<00:15, 23.80it/s]

zero-size array to reduction operation maximum which has no identity


100%|██████████| 3/3 [00:00<00:00, 61.43it/s]


In [19]:
from malaya_speech.utils import tf_featurization

config = malaya_speech.config.transducer_featurizer_config
featurizer = tf_featurization.STTFeaturizer(**config)

In [20]:
n_mels = 80

def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    example['inputs'] = mel_fbanks
    return example

def parse(serialized_example):

    data_fields = {
        'waveforms': tf.compat.v1.VarLenFeature(tf.float32),
        'targets': tf.compat.v1.VarLenFeature(tf.int64),
        'targets_length': tf.compat.v1.VarLenFeature(tf.int64),
    }
    features = tf.compat.v1.parse_single_example(
        serialized_example, features = data_fields
    )
    for k in features.keys():
        features[k] = features[k].values
        
    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['waveforms', 'inputs', 'targets', 'targets_length']:
            features.pop(k, None)

    return features

def get_dataset(files, batch_size = 2, shuffle_size = 32, thread_count = 24):
    def get():
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.repeat()
        return dataset

    return get

In [21]:
files = tf.io.gfile.glob('gs://mesolitica-tpu-general/imda/part4-same-room/*.tfrecord')
d = get_dataset(files)()
d = d.as_numpy_iterator()

In [23]:
next(d)

{'targets': array([ 69, 108, 396, 795,  65,   6,  20, 489, 375, 795, 464,   9, 223,
         37, 140, 880]),
 'targets_length': array([16]),
 'waveforms': array([-0.01147776, -0.01396093, -0.01743737, ...,  0.00623551,
         0.00469043,  0.00469043], dtype=float32),
 'inputs': array([[-1.1467438 , -0.9835571 , -0.7878138 , ..., -2.358215  ,
         -1.7385253 , -1.9502212 ],
        [-1.3655019 , -1.3327678 , -1.2868116 , ..., -1.8851404 ,
         -1.9457333 , -2.0116887 ],
        [-1.4144297 , -1.1720926 , -0.9228275 , ..., -1.595703  ,
         -1.831306  , -1.9690977 ],
        ...,
        [-1.1878959 , -1.076973  , -0.92887765, ..., -1.5371672 ,
         -1.7004726 , -2.0543156 ],
        [-1.1862378 , -1.1690313 , -1.1501343 , ..., -1.3770837 ,
         -1.6801057 , -1.781006  ],
        [-1.2420168 , -1.1447793 , -1.0114012 , ..., -1.7152041 ,
         -1.9377683 , -1.8452353 ]], dtype=float32)}